## Introduction 

Today we will be plotting and fitting a dose-response curve. The data that we will be working with are whole-cell electrophysiology recordings made from acute brain slices from rats, an example trace is shown below. We will be plotting the current responses to different concentrations of the mu-opioid receptor (MOR) agonist morphine for males versus females. The data consists of multiple experiments for each concentration. 

When MORs are activated they in turn activate G-protein Inwardly-Rectifying Potassium channels (GIRKs), which is what the current responses below are actually measuring. These channels allow potassium to leave the cell, and this is what creates the electrical current that we are measuring. Because there could be variable numbers of GIRK channels on a cell, the morphine current responses will be normalized to the current responses induced by a saturating concentration of the alpha2-adrenergic receptor agonist UK, which also acts on GIRKs. Because UK is very potent, it will give us the maximum GIRK response possible. 

I have already manually calculated the current responses for each drug so the file we will be working with is a csv file of these values.

Below is an example trace with morphine application, block with the MOR antagonist naloxone (NLX), and then application of UK followed by the appriopriate (alpha2) antagonist, ida. 

<img src="trace1.png" width = "800" >

## Loading the data 

First, we need to import the modules we will need and load the data. We have seen all of these before except for the `curve_fit` function from [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), which will allow us to fit a curve to our dose response plot.  

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np

pd.options.display.max_rows = 7

### Exercise

Next, we need to import the file that we'll be working with, which is `dose_response_male.csv`. Please first define a string variable called `dose_response_male` that contains the file name. Next, import this file using `pd.read_csv`. This file is organized so that the first column is the concentration, this will be an appropriate index so set this first column as the index column. 

In [ ]:
# Answer
filename = "dose_response_male.csv"

dose_response_male = pd.read_csv(filename, index_col=0)
dose_response_male

## Working with our data

### Exercise

Great, now we have loaded our dataframe and can work with it. Later on when we fit a dose response curve we will need the log values of each concentration. So create a new column in our data frame titled `'Log[M]'` that contains the log values. 

Hints
* I included a column in the dataframe that contains each concentration in Molar.
* The function you need for computing the log is part of the Numpy package, which we imported as `np`. Many Numpy functions can also work with Pandas DataFrames and Series, not just Numpy arrays.

In [ ]:
# Answer
dose_response_male['Log[M]'] = np.log10(dose_response_male['concentration_in_M'])
dose_response_male

### Exercise

Now, as I mentioned, we will be plotting normalized values. The morphine current will be plotted as a percentage of the UK current. So let's add another column to our dataframe titled `'percent_UK'` that contains the normalized values. 

In [ ]:
# Answer 
dose_response_male['percent_UK'] = 100 * dose_response_male['morphine_current'] / dose_response_male['UK_current']
dose_response_male

### Exercise

Our data contains multiple replicates for each concentration. However, we want to plot the average normalized current for each concentration. How might we group (hint) the data based on concentration and then calculate the average? Store your final answer in the variable `mean_responses_male`.

In [ ]:
# Answer 
grouping_male = dose_response_male.groupby(['Log[M]'])
mean_responses_male = grouping_male['percent_UK'].mean()
mean_responses_male

## Plotting our data

### Exercise

Now let's plot our data using `matplotlib.pyplot`! The default is that there will be a line connecting the data, but we want to fit our own line later on. To do this you will need to add an argument for the type of `marker` to use for each point as well as an argument to change the default `linestyle`. Use `plt.plot?` for help in doing this and for a list of markers you could use. Hint: look under "Other Parameters". Don't forget to add labels for each axis and a plot title.

You've already worked with plotting a little. For example, you've generated a plot using the method available on DataFrames. Internally, this method uses `matplotlib.pyplot` to do most of the work. Today, we're generating the plot ourselves using `matplotlib.pyplot` directly. The function, `plt.plot` is designed to work with various types of data. *Usually* you would provide both x and y values to the function. However, [`plt.plot` can work with *lableled* data](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html). Where have we seen labeled data?  Pandas Series are one of them. The *index* of a series are the labels for the values. In this case, `plt.plot` is smart enough to use the series values for the y-values and the labels (i.e., the index) for the x-values.


In [ ]:
# Answer 
plt.plot(mean_responses_male, marker='o', linestyle='')
plt.xlabel('Log[M]')
plt.ylabel('% UK')
plt.title('Morphine Dose Response')

### Exercise

Great, now lets add some error bars. First let's calculate the standard error of the mean and store them in the variable `sem_responses_male`. Hint: is there a dataframe method that can do this for us? <font>

In [ ]:
# Answer 
sem_responses_male = grouping_male['percent_UK'].sem()
sem_responses_male

### Exercise 

Now let's add error bars to our graph. To do this, you can use `plt.errorbar` instead of `plt.plot` because `plt.errorbar` will plot the data points associated with the errorbars as well. 

Note that `plt.errorbar` takes two necessary arguments for the x-values and y-values. Earlier, we used `plt.plot`, but did not specify the x-values. Instead, `plt.plot` was able to infer the x-values using the labels from the Pandas Series, `mean_responses_male`. Why isn't `plt.errorbar` smart enough to do the same? It's because the developers of this librarly chose not to implement this feature (or have not thought to do so yet). So, we need to do a little more work (i.e., typing) to specify the x-values and y-values.

We have the y-values stored in `mean_responses_male`, so that's easy, but what do we use for the x-values? The grouping we made before has an index made up of the Log[M] concentrations, so we can use `mean_responses_male.index.values` as the x-values! This uses two attributes from Pandas. The `index` attribute retreives the index values from `mean_responses_male` and the `values` attribute returns an array of these index values.

Also, the default for error bars is to not have a cap on them. Let's add some. Hint: what argument(s) does `plt.errorbar` take that will help us do this? 

In [ ]:
x_data = mean_responses_male.index.values

# Answer 
plt.errorbar(x_data, mean_responses_male, yerr=sem_responses_male, marker='o', linestyle='', capsize=3)
plt.xlabel('Log[M]')
plt.ylabel('% UK')
plt.title('Morphine Dose Response')
plt.show()

## Fitting a Curve 

### Exercise

Now we will fit a a curve to the data. To do this, we need to have the x-values and the y-values as separate Numpy arrays. We already have the x-values as an array from when we plotted the errorbars above. So now let's deal with the y-values... What are the y-values in this dataset? Create a numpy array of these values and store it in the variable `y_data_male`.

In [ ]:
# Answer 
y_data_male = mean_responses_male.values
y_data_male

**Question** How many dimensions does this array have? What's the shape?

### Exercise

OK, now we can fit the data! But with what equation? A very important step in curve fitting is knowing what curve to fit to your data, and knowing enough about your system/experiment to know what sort of models (or curves) fit. We will give you the equation for this dataset, but your own dataset may be different, and may require a different fit equation.

For this data (and dose-response curves in general), the [Hill equation](https://en.wikipedia.org/wiki/Hill_equation_(biochemistry)) is appropriate. It is commonly used to fit a variety of dose-response curves in biological and biochemical systems. The Hill equation has four parameters (i.e., constants) that need to be fit to our data:

-  `EC50` is the concentration of agonist that gives a response half way between Bottom and Top. 
-  `Top` and `Bottom` are plateaus in the units of the Y axis.
-  The equation we will use assumes that the dose response curve has a standard slope, equal to a `hillslope` (or slope factor) of 1.0

<img src='hill_eq.png' width='500' >

What is the value of using `curve_fit`? Why could we just not guess? Well, let's try to guess the arguments (i.e., the values of the parameters) for the equation and see how good we are... We have entered some guesses for you to try. For this exercise, you need to:

* Write code that calculates the y-values using the Hill equation above
* Plot the resulting curve
* Plot the actual data on top f the curve

Hint, the code for calculating the y-values should start with:

    y = bottom + ...

**Note** We are not writing a function to do this (yet). That will be in the next exercise.

In [ ]:
x = x_data
bottom = 10
top = 60
EC50 = 3e-7
hillslope = 1 

# Answer 
y = bottom + (top-bottom)/(1 + 10**(np.log10(EC50)-x)) 
plt.plot(x_data, mean_responses_male, "o")
plt.plot(x, y)

### Exercise

Before we use `curve_fit`, we need to define a function for the Hill equation. This will be called by the `curve_fit` function later. The function we will write takes parameters `x`, and all of the constants in the equation we are using `bottom, top, EC50, hillslope`. 

**Note** By convention, the scipy `curve_fit` function we are using wants `x` as the first parameter.

In [ ]:
# Answer 
def hill_eq(x, bottom, top, EC50, hillslope):
    y = bottom + (top-bottom)/(1+10**(np.log10(EC50)-x))
    return y 

### Exercise

Great! This function can be used to make guesses just like above. Try using it with the numbers below and then plot them again. 

In [ ]:
# bottom = 5, top = 60, EC50 = 8e-8, hillslope = 1
# Answer 
y = hill_eq(x, 5, 60, 8e-8, 1)
plt.plot(x,y)
plt.plot(x_data, mean_responses_male, "o")

Now, try adjusting the parameters one or two times in the cell above. Can you get a line that looks like a better fit? 

How do we evaluate goodness of fit? A common method is the least-squares approach. First, we compute the difference between the *observed* y-values (i.e., `mean_responses_male`) and *predicted* y-values (i.e., `y`). For this exercise: 

* Compute the difference between `mean_responses_male` and `y` and save it in a variable called `residuals`.
* Take the square of `residuals` and sum all the values in this array. Save the resulting number as `sum_of_squares`.

Look at how the sum of squares changes as you play with the parameters for the hill equation.

In [ ]:
y = hill_eq(x, 5, 60, 8e-8, 1)
plt.plot(x_data, mean_responses_male, "o")
plt.plot(x, y)

# Answer 
residuals = mean_responses_male - y
sum_of_squares = np.sum(residuals**2)
sum_of_squares

This line looks a lot closer but using `curve_fit` is still more valuable for reasons that we will describe below.

### Exercise

Now let's use scipy's `curve_fit` function, which we imported from `scipy.optimize` earlier. `curve_fit` returns two arrays- popt and pcov- that contain the optimal parameters and covariance, respectively. We mostly care about popt, as those are the constants we want to describe the curve (`bottom, top, EC50, hillslope`). These can be named whatever you want them to be, but you will need to set two variables when calling the `curve_fit` function, the first will store the popt values and the second the pcov values.

We will also use bounds to limit the constants to positive numbers within a reasonable range. They need to be limited to positive numbers because you can't take the log of a negative number. If `curve_fit` tries to use a negative for EC50, you will get an error. We used 100 here because by visualizing the data, we know the values of each of our constants could not be greater than 100.

See if you can figure out how to construct the curve fit statement. Store the values `curve_fit` returns in the variables `male_fit` and `pcov_m`. Use `curve_fit?` for help writing the statement.

In [ ]:
bounds = (0, 100) # Limits fit to only positive values, within a reasonable range

# Answer
male_fit, _ = curve_fit(hill_eq, x_data, y_data_male, bounds=bounds)
male_fit

## Plotting the curve fit

### Exercise

OK, now let's plot the curve using `matplotlib.pyplot`.

To make our lives easier, we can use `*` to help us. `*` automatically unpacks arguments from a list, tuple, or array:

    def my_function(a, b, c, d):
        print('{} {} {} {}'.format(a, b, c, d))
        
    a = 1
    other_args = [2, 3, 4]
    my_function(a, *other_args)
    
Note that you cannot do:

    my_function(*other_args)
    
Why? What about this? would it work?

    args = [1, 2, 3, 4]
    my_function(*args)
    
Would this work?

    args = [1, 2, 3, 4]
    my_function(4, *args)
    
Would this work?

    args = [1, 2, 3]
    my_function(*args, 4)

Because popt is an array of our fit constants, we can use `*male_fit` to pass the arguments from the array to `bottom, top, EC50, hillslope`. To plot the line, first set `male_line` equal to the `hill_eq` function with the optimal paramaters we just calculated (`male_fit`). Then plot using `plt.plot`.

In [ ]:
# Answer 
male_line = hill_eq(x_data, *male_fit)
plt.plot(x_data, male_line)

In [ ]:
# Isn't that much easier than...
male_line = hill_eq(x_data, male_fit[0], male_fit[1], male_fit[2], male_fit[3])
plt.plot(x_data, male_line)

### Exercise 

Now we will add the data we plotted earlier to visualize how good our fit is. Don't forget your labels! You can specify color in many ways including [HTML color names](https://www.w3schools.com/colors/colors_names.asp).

In [ ]:
plt.plot(x_data, male_line, color='deepskyblue')

# Answer 
plt.errorbar(x_data, mean_responses_male, yerr=sem_responses_male, marker='o', linestyle='', capsize=3, color='blue')
plt.xlabel('Log[M]')
plt.ylabel('% UK')
plt.title('Morphine Dose Response')
plt.show()

Great this looks good! But why is the line with the constants `curve_fit` found better than the one with our guesses? Curve fitting works by optimizing the four parameters in order to minimize the sum of squared errors between the fit line and the observed values.

Run the code below to see how much better the line using curve_fit is compared to our guesses (lower is better). `curve_fit` automates the process of finding the arguments for `hill_eq` that result in a prediction with the lowest possible sum of squared errors. 

In [ ]:
# y contains the prediction from our last guess
sum_sq_errors = np.sum((mean_responses_male - y)**2)
print("Sum of sq errors (our guess): {}".format(sum_sq_errors))

# male_line contains the prediction from curve_fit
best_fit_errors = np.sum((mean_responses_male - male_line)**2)
print("Sum of sq errors (curve_fit): {}".format(best_fit_errors))

## Adding the female dose response (bonus exercises)

Now let's do the same thing for the female dose response. Just run the code below (it repeats everything we did above).

In [ ]:
# Load the file 
dose_response_female = pd.read_csv('dose_response_female.csv', index_col=0)

# Create a column of normalized values 
dose_response_female['percent_UK'] = 100 * dose_response_female['morphine_current'] / dose_response_female['UK_current']

# Create a column of log values 
dose_response_female['Log[M]'] = np.log10(dose_response_female['concentration_in_M'])

# Average based on concentration 
grouping_female = dose_response_female.groupby(['Log[M]'])
mean_responses_female = grouping_female['percent_UK'].mean()

# Calculate SEM 
sem_responses_female = grouping_female['percent_UK'].sem()

# Create an array of our y-values 
y_data_female = mean_responses_female.values

### Exercise 

We can use the same function we defined above for the hill equation to again find the optimal parameters in order to fit the curve for the famale dose-response curve. This time use the variables `female_fit` and `pcov_f`. 

In [ ]:
# Answer 
female_fit, pcov_f = curve_fit(hill_eq, x_data, y_data_female, bounds=bounds)
female_fit

### Exercise 

Now let's plot the mean responses for males and females along with their curve fits all on the same plot! And let's add a legend this time. 

In [ ]:
# Answer
female_line = hill_eq(x_data, *female_fit)

plt.plot(x_data, male_line, color='deepskyblue')
plt.plot(x_data, female_line, color='plum')
plt.errorbar(x_data, mean_responses_male, yerr=sem_responses_male, marker='o', linestyle='', capsize=3, color = 'blue')
plt.errorbar(x_data, mean_responses_female, yerr=sem_responses_female, marker='o', linestyle='', capsize=3, color='purple')
plt.xlabel('Log[M]')
plt.ylabel('% UK')
plt.title('Morphine Dose Response')
plt.legend(['Male Fit', 'Female Fit', 'Male', 'Female'])
plt.show()

## Working with the curve_fit constants

### Exercise 

Now we have a beautiful plot, but it would be nice to know what values were used to fit the curve. These values are stored in  the popt array, but let's convert them to a series to make them more easily readable. First make a list of the labels (`['bottom', 'top', 'EC50 (M)', 'hill slope']`) and then use `pd.Series` to make a series with an index of the fit labels from your list. Do this for both males and females.

In [ ]:
# Answer 
fit_labels = ['bottom', 'top', 'EC50 (M)', 'hill slope']
fit_series_m = pd.Series(male_fit, index = fit_labels)
fit_series_f = pd.Series(female_fit, index = fit_labels)
print("Males: \n" + str(fit_series_m))
print("\nFemales: \n" + str(fit_series_f))

To make the EC50 values easier to read we can convert it to nM and then print out a statement that rounds it to two decimal points:

In [ ]:
ec50_f = fit_series_f['EC50 (M)'] * 1e9
ec50_m = fit_series_m['EC50 (M)'] * 1e9
print("The EC50 for females is {:.2f} nM".format(ec50_f))
print("The EC50 for males is {:.2f} nM".format(ec50_m))